<a href="https://colab.research.google.com/github/lucasnserra/Audio2Text-Sentiment-Analyst/blob/main/Analisis_de_sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Esta Notebook utiliza 2 modelos preentrenados de Hugging Face basados en transformers que te permitiran pasar audios a texto y tambien analizar la probabilidad que ese texto sea Positivo, negativo o neutro.


In [69]:
#instalamos librerias necesarias 
!pip install transformers 
!pip install pysentimiento
!pip install soundfile
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
#importamos librerias que consideramos necesarias para nuestro analisis

#librerias basicas
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import requests

#importamos librerias de analisis de sentimiento
from pysentimiento import create_analyzer #analyzer = create_analyzer(task="sentiment", lang="es")

#Libreria de procesamiento de audio a texto 
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch
import soundfile as sf
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration



In [71]:
#Funcion de analisis de sentimiento
def prediccion_texto(texto):
    analyzer = create_analyzer(task="sentiment", lang="es")
    resultado = analyzer.predict(texto)
    return resultado

In [72]:
#Prueba de funcion de analisis de texto
resultado_bueno = prediccion_texto("Este es un texto de ejemplo, el cual es muy bueno para predecir")
resultado_malo = prediccion_texto("Este es un texto de ejemplo, el cual es muy malo para predecir")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--pysentimiento--robertuito-sentiment-analysis/snapshots/12e030859ce19539e24b486ac84ffebb9b68ecf1/config.json
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_

In [73]:
#probamos resultados
print("Resultado bueno " + str(resultado_bueno), "Resultado Malo " + str(resultado_malo))


Resultado bueno AnalyzerOutput(output=POS, probas={POS: 0.941, NEU: 0.056, NEG: 0.003}) Resultado Malo AnalyzerOutput(output=NEG, probas={NEG: 0.970, NEU: 0.028, POS: 0.002})


In [74]:
# cargo y proceso el modelo de audio a texto
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

#Defino funcion para transcribir el audio
def transcribir_audio(audio_file):
    # load audio file and extract features
    audio, sr = librosa.load(audio_file, sr=processor.feature_extractor.sampling_rate)
    input_features = processor(audio, sampling_rate=sr, return_tensors="pt").input_features 
    
    # generate token ids
    predicted_ids = model.generate(input_features)
    
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    
    return transcription[0]



Streaming output truncated to the last 5000 lines.
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0
    ],
    [
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
  

In [75]:
def descargar_audio(url, audio):
    response = requests.get(url)

    with open(audio, 'wb') as f:
        f.write(response.content)

In [76]:
#Leo archivo de audio para la prueba 1
url = 'https://github.com/lucasnserra/Audio2Text-Sentiment-Analyst/raw/0c038956cc6eefdf69674ccdd520a368870621e0/audio.wav'
response = requests.get(url)

with open('audio.wav', 'wb') as f:
    f.write(response.content)

audio = 'audio.wav'


critica = transcribir_audio(audio)
critica

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


' Clicica rápida de Wendy de Poo, Blood and Coney. No esperes una película de sentenía una buena historia, ya que no tiene absolutamente nada nuevo que contar. Sus actuaciones son malas y en sí es una película generica del género Slasher. Lo único que puede ser algo bueno es que te puedes sacar una que otra risa por algunos de sus momentos o diálogoza sordos. Calificación final, 2.5 y si quieres ver mi crítica completa, suscríbete a mi canal, sin atulado.'

In [77]:
#Leo archivo de audio para la prueba Nro 2
url2 = 'https://github.com/lucasnserra/Audio2Text-Sentiment-Analyst/blob/main/audio2.wav?raw=true'
response2 = requests.get(url2)

with open('audio.wav', 'wb') as f:
    f.write(response2.content)

audio2 = 'audio.wav'
critica_ = transcribir_audio(audio2)
critica_

' Cripticas y en spoilers de Arman y la Vista a Cuantumania, Kang es el alma de Arman y la Vista. El conquistador por fin hace acto de presencia y quien el protagonismo que se merece, con un llor a tan mayor que parece haber nacido para este papel. La gran sorpresa para esta película ha sido modo con un personaje del que nos esperaban nada con secuencias brutalmente divertidas. Tarkomiendo que te quede esa hasta el final de la película porque cuenta con dos escenas post créditos geniales, ambas tienen repercusiones para el futuro.'

In [78]:
#Leo archivo de audio para la prueba Nro 2
url2 = 'https://github.com/lucasnserra/Audio2Text-Sentiment-Analyst/blob/main/audio2.wav?raw=true'
response2 = requests.get(url2)

with open('audio2.wav', 'wb') as f:
    f.write(response2.content)

audio2 = 'audio2.wav'
critica_ = transcribir_audio(audio2)
critica_

' Cripticas y en spoilers de Arman y la Vista a Cuantumania, Kang es el alma de Arman y la Vista. El conquistador por fin hace acto de presencia y quien el protagonismo que se merece, con un llor a tan mayor que parece haber nacido para este papel. La gran sorpresa para esta película ha sido modo con un personaje del que nos esperaban nada con secuencias brutalmente divertidas. Tarkomiendo que te quede esa hasta el final de la película porque cuenta con dos escenas post créditos geniales, ambas tienen repercusiones para el futuro.'

In [79]:
#Leo archivo de audio para la prueba Nro 3
url3 = 'https://github.com/lucasnserra/Audio2Text-Sentiment-Analyst/blob/main/audio3.wav?raw=true'
response3 = requests.get(url3)

with open('audio3.wav', 'wb') as f:
    f.write(response3.content)

audio3 = 'audio3.wav'
critica_3 = transcribir_audio(audio3)
critica_3

' Venía prometiendo que va a ser la saga revolucionaria del cine y que nos iba a plantar cada para poder ver si nos hacía dónde van las películas y la verdad es que nos encontramos con un tremendo aburrimiento de trama. Mas allá del show espectacular que son. No, la pena yo. No, no. No sé si no vale la pena el show o sea si vas a ver una película yo creo que no vale la pena.'

In [80]:
prediccion = prediccion_texto(critica)
prediccion2 = prediccion_texto(critica_)
prediccion3 = prediccion_texto(critica_3)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--pysentimiento--robertuito-sentiment-analysis/snapshots/12e030859ce19539e24b486ac84ffebb9b68ecf1/config.json
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_

In [81]:
print('El resultado de la primera prediccion es ' + prediccion.output.replace('NEU', 'Neutro').replace('POS', 'Positivo').replace('NEG', 'Negativo') + ' con las siguientes probabidades ' + str(prediccion.probas))

El resultado de la primera prediccion es Neutro con las siguientes probabidades {'NEG': 0.20106437802314758, 'NEU': 0.4933183491230011, 'POS': 0.3056173324584961}


In [82]:
print('El resultado de la segunda prediccion es ' + prediccion2.output.replace('POS', 'Positivo').replace('NEU', 'Neutro').replace('NEG', 'Negativo') + ' con las siguientes probabidades ' + str(prediccion2.probas))

El resultado de la segunda prediccion es Positivo con las siguientes probabidades {'NEG': 0.008234702050685883, 'NEU': 0.05927221477031708, 'POS': 0.9324930906295776}


In [83]:
print('El resultado de la tercera prediccion es ' + prediccion3.output.replace('POS', 'Positivo').replace('NEU', 'Neutro').replace('NEG', 'Negativo') + ' con las siguientes probabidades ' + str(prediccion3.probas))

El resultado de la tercera prediccion es Negativo con las siguientes probabidades {'NEG': 0.9475676417350769, 'NEU': 0.04628613591194153, 'POS': 0.00614620465785265}
